In [8]:
import os
import sys
from pathlib import Path

from sklearn.linear_model import SGDClassifier

# Dynamically locate project root and set working dir
project_root = Path(__file__).resolve().parent.parent.parent.parent
os.chdir(project_root)
sys.path.append(str(project_root))
print(os.getcwd())

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from catboost import CatBoostRegressor, Pool
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingRandomSearchCV
from typing import Union, Dict, Tuple
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import SGDRegressor
from sklearn.multioutput import MultiOutputRegressor
from ydata_profiling import ProfileReport
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor as RF, RandomForestRegressor
import warnings
import optuna
warnings.filterwarnings('ignore')
%matplotlib inline

NameError: name '__file__' is not defined

# Table of Contents
- [Dataset Preparation](#Dataset-Preparation)
- [Baseline Model](#Baseline-Model)
- [Baseline Model with target lag features](#Baseline-Model-with-Target-Lag-Features)
- [Baseline Model with all lag features](#Baseline-Model-with-All-Lag-Features)
- [Gradient Boosting with Hyperparameters Tuning](#Gradient-Boosting-with-Hyperparameters-Tuning)
- [Conclusions](#Conclusions)

# Dataset Preparation

In [5]:
# Loading the data
#data = pd.read_parquet('ml-project-blueprint/ml-project-blueprint/data/raw_data/Mendeley_data/raw_data.parquet.gzip',
#filters=[('2-PAT control(PAT_ref:PAT ref)', '=', 5)],columns=['Batch reference(Batch_ref:Batch ref)'])

df = pd.read_csv('./data/raw_data/Mendeley_data/100_Batches_IndPenSim_V3.csv',
                 usecols=list[np.arange(1,50,1)])
data = df.rename(columns={'2-PAT control(PAT_ref:PAT ref)': 'Batch reference(Batch_ref:Batch ref)','Batch reference(Batch_ref:Batch ref)':'2-PAT control#(PAT_ref:PAT ref)'})


NameError: name 'pd' is not defined

In [12]:
data.columns[:40] # we are looking at all the process metrics upto the raman spectroscopy reading at 2400 mm

Index(['2-PAT control#(PAT_ref:PAT ref)'], dtype='object')

In [5]:
data[data.columns[:40]].describe()

,Time (h),Aeration rate(Fg:L/h),Agitator RPM(RPM:RPM),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),...,Viscosity(Viscosity_offline:centPoise),Fault reference(Fault_ref:Fault ref),0 - Recipe driven 1 - Operator controlled(Control_ref:Control ref),1- No Raman spec,1-Raman spec recorded,Batch reference(Batch_ref:Batch ref),2-PAT control#(PAT_ref:PAT ref),Batch ID,Fault flag,2400
count,113935.000000,113935.000000,113935.0,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,...,2062.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000,113935.000000
mean,114.750656,65.246360,100.0,76.663764,0.073209,61.334389,74.346341,20.763025,154.811954,0.945026,...,51.546454,0.011024,0.301795,1.295783,50.402466,50.402466,0.101154,89307.003616,89402.127915,89663.428859
std,66.990504,11.690215,0.0,25.680134,0.552788,44.972713,108.022600,50.230266,155.601474,0.134269,...,24.073778,0.104415,0.459039,0.456396,28.862140,28.862140,0.301534,47765.751082,47852.683115,47941.881983
min,0.200000,20.000000,100.0,2.000000,0.000000,0.000000,0.000100,0.000100,0.000000,0.600000,...,4.075300,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000
25%,57.000000,60.000000,100.0,72.000000,0.000000,35.766000,11.157000,0.000100,0.000000,0.900000,...,34.812250,0.000000,0.000000,1.000000,26.000000,26.000000,0.000000,47016.500000,47029.500000,47208.500000
50%,114.000000,65.000000,100.0,80.000000,0.000000,55.407000,34.384000,0.159010,100.000000,0.900000,...,53.154000,0.000000,0.000000,1.000000,50.000000,50.000000,0.000000,89699.000000,89772.000000,90038.000000
75%,171.000000,75.000000,100.0,90.000000,0.000000,76.271500,94.904500,11.640500,250.000000,1.100000,...,73.536000,0.000000,1.000000,2.000000,76.000000,76.000000,0.000000,129390.000000,129570.000000,129920.000000
max,290.000000,75.000000,100.0,150.000000,12.996000,225.000000,1500.000000,1500.000000,500.000000,1.100000,...,117.930000,1.000000,1.000000,2.000000,100.000000,100.000000,1.000000,194630.000000,195340.000000,196100.000000


In [4]:
# saving the batch references for later use

batch_references = data['Batch reference(Batch_ref:Batch ref)'] 



We are going to work with the columns identified in the EDA notebook as the input and target parameters respectively.



In [5]:
target_features = ['Offline Biomass concentratio(X_offline:X(g L^{-1}))',
                  'Offline Penicillin concentration(P_offline:P(g L^{-1}))',
                ]

input_features = ['carbon dioxide percent in off-gas(CO2outgas:%)','Oxygen in percent in off-gas(O2:O2  (%))',
                  'Dissolved oxygen concentration(DO2:mg/L)','Vessel Weight(Wt:Kg)','pH(pH:pH)','Temperature(T:K)','2400',
                  'Aeration rate(Fg:L/h)','PAA flow(Fpaa:PAA flow (L/h))','2400',
                 'Time (h)','Batch reference(Batch_ref:Batch ref)'
                 ]


In [6]:
df_grouped =  pd.DataFrame(data.groupby('Time (h)')[input_features].mean()) # here we are taking all the attributes until column '2400'.
df_grouped.dropna()

,carbon dioxide percent in off-gas(CO2outgas:%),Oxygen in percent in off-gas(O2:O2 (%)),Dissolved oxygen concentration(DO2:mg/L),Vessel Weight(Wt:Kg),pH(pH:pH),Temperature(T:K),2400,Aeration rate(Fg:L/h),PAA flow(Fpaa:PAA flow (L/h)),2400,Time (h),Batch reference(Batch_ref:Batch ref)
Time (h),,,,,,,,,,,,
0.2,0.086481,0.204688,14.69160,61954.14,6.505869,297.6942,0.0,30.0,5.0,0.0,0.2,50.5
0.4,0.097858,0.206046,14.67345,61961.69,6.514475,297.8896,0.0,30.0,5.0,0.0,0.4,50.5
0.6,0.101635,0.206280,14.65668,61970.24,6.526232,297.8675,0.0,30.0,5.0,0.0,0.6,50.5
0.8,0.103909,0.206216,14.63790,61978.02,6.536673,298.1383,0.0,30.0,5.0,0.0,0.8,50.5
1.0,0.105942,0.206058,14.61566,61983.71,6.539947,298.0328,0.0,30.0,5.0,0.0,1.0,50.5
...,...,...,...,...,...,...,...,...,...,...,...,...
289.2,1.416300,0.189250,8.94060,75756.00,6.505800,297.9200,195300.0,60.0,4.0,195300.0,289.2,29.0
289.4,1.417200,0.189110,8.85360,75806.00,6.502200,298.2100,195280.0,60.0,4.0,195280.0,289.4,29.0
289.6,1.418200,0.189060,8.82760,75855.00,6.498600,298.0700,195260.0,60.0,4.0,195260.0,289.6,29.0


In [7]:
data.head(3)

,Time (h),Aeration rate(Fg:L/h),Agitator RPM(RPM:RPM),Sugar feed rate(Fs:L/h),Acid flow rate(Fa:L/h),Base flow rate(Fb:L/h),Heating/cooling water flow rate(Fc:L/h),Heating water flow rate(Fh:L/h),Water for injection/dilution(Fw:L/h),Air head pressure(pressure:bar),...,210,209,208,207,206,205,204,203,202,201
0,0.2,30,100,8,0.0,30.118,9.8335,0.0001,0,0.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
1,0.4,30,100,8,0.0,51.221,18.1550,0.0001,0,0.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
2,0.6,30,100,8,0.0,54.302,9.5982,0.0001,0,0.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN


# Baseline Model

We will use Random Forest using Raw Features as the Baseline Model because it does not require much tuning.

Then, we will add features and try other algorithms to see how big the improvement is.

For the metrics, we will use:
- MAPE
- MAE
- RMSE

In [8]:
def compute_metrics(
    y_true: Union[np.ndarray, list], 
    y_pred: Union[np.ndarray, list],
    target: str
) -> Dict[str, float]:
    """
    Compute evaluation metrics between true and predicted values.

    Metrics returned:
    - MAPE: Mean Absolute Percentage Error (in %)
    - MAE: Mean Absolute Error
    - RMSE: Root Mean Squared Error

    Parameters:
    ----------
    y_true : array-like
        Ground truth values.
    y_pred : array-like
        Predicted values.

    Returns:
    ------
    dict
        Dictionary with keys 'MAPE', 'MAE', and 'RMSE' and their float values.
    """
    y_true = y_true[target]
    #y_pred = y_pred[target]
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred, squared=False))

    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    mape = np.mean(np.abs((y_true - y_pred) / np.where(y_true == 0, 0.01, y_true))) * 100

    return {
        'target':target,
        'MAE': round(mae, 2),
        'RMSE': round(rmse, 2),
        'MAPE': round(mape, 2)
    }

In [9]:
def prepare_dataset(
    df: pd.DataFrame, 
    target_columns: list,
    input_columns: list,
    train_fraction: float = 0.8,
) -> Tuple[pd.DataFrame, pd.DataFrame, pd.Series, pd.Series]:
    """
    Splits a DataFrame into training and testing sets for features and target.

    Parameters:
    ----------
    df : pd.DataFrame
        The input DataFrame that must contain a 'target' column.
    train_fraction : float, optional (default=0.8)
        The fraction of data to use for training (between 0 and 1).

    Returns:
    -------
    x_train : pd.DataFrame
        Training features.
    x_test : pd.DataFrame
        Testing features.
    y_train : pd.Series
        Training target values.
    y_test : pd.Series
        Testing target values.
    """
    feats = [col for col in df.columns if col in input_columns]
    x, y = df[feats], df[target_columns]
    train_size = int(train_fraction * df.shape[0])
    x_train, x_test = x[:train_size], x[train_size:]
    y_train, y_test = y[:train_size], y[train_size:]
    return x_train, x_test, y_train, y_test

In [10]:
def visualize_features(features:list,dataframe:pd.DataFrame):

    for j in features:
        fig = go.Figure()

        yaxis_title = j





# Add trace
        fig.add_trace(go.Scatter(
            x=dataframe.index,
            y=dataframe[j],
            mode="lines",  # or "lines+markers
            name="y value",

    ))

        fig.update_layout(
            width=1000,    # width in pixels
            height=400,   # height in pixels
            xaxis_title='Time in 12 Minutes Intervals',
            yaxis_title=yaxis_title

    )
        fig.show()


#### Target definition

- ##### As for the targets, we have 5 in total we could pursue. I am going to first build a baseline model for the targets and evaluate the resulting scores, and the strategy is to push forward for now, the targets having the highest 2 or 3 metrics.

- ##### First of all, we need to impute the missing measurements of these targets, as they were measured in 12h intervals.

- ##### I have chosen the strategy of spline interpolation, as the more simpler methods of imputing a statistical property of the target variables for the NA values would distort the target variable strongly.



In [11]:
df = data[data.columns[:40]].copy()



#### Imputation of missing data in target variables

In [12]:
df.isna().sum()

Time (h)                                                                   0
Aeration rate(Fg:L/h)                                                      0
Agitator RPM(RPM:RPM)                                                      0
Sugar feed rate(Fs:L/h)                                                    0
Acid flow rate(Fa:L/h)                                                     0
Base flow rate(Fb:L/h)                                                     0
Heating/cooling water flow rate(Fc:L/h)                                    0
Heating water flow rate(Fh:L/h)                                            0
Water for injection/dilution(Fw:L/h)                                       0
Air head pressure(pressure:bar)                                            0
Dumped broth flow(Fremoved:L/h)                                            0
Substrate concentration(S:g/L)                                             0
Dissolved oxygen concentration(DO2:mg/L)                                   0

In [13]:
# interpolating target features

interpolated_targets = []

for i in target_features:
    
    df[f'{i}_interpolated'] = df[i].interpolate(option='spline')
    df[f'{i}_interpolated'] = df[f'{i}_interpolated'].fillna(method='bfill')
    interpolated_targets.append(f'{i}_interpolated')




In [14]:
data_grouped = df.groupby('Time (h)').mean() # grouping of all the batches to get an overview of the behaviour over time

visualize_features(interpolated_targets, data_grouped)



##### Let's visualize the input features one by one, to get a general overview of how they behave with time.

In [15]:
data_grouped = df.groupby('Time (h)').mean() # grouping of all the batches to get an overview of the behaviour over time


visualize_features(input_features[:-2], data_grouped)



- ##### Here we can see that there are two distinct groups of features. One of them display a certain trend with time, with visible cyclic patterns in the 25-50 hour time interval unit. The other group, namely pH and Temperature, display a highly cyclic pattern within a much shorter time interval.

- ##### Let's zoom on pH and Temperature further

In [16]:


visualize_features(['pH(pH:pH)','Temperature(T:K)'], data_grouped)


- ##### Looking at these 2 features in detail, we can conclude that they are becoming highly cyclic from the 150h mark, of the batch run.
- ##### This means we would need to handle feature engineering differently for them instead of lags or moving averages, we are going to take the moving standard deviation and reenginer these features in a sine-cosine format.

In [17]:
input_features

['carbon dioxide percent in off-gas(CO2outgas:%)',
 'Oxygen in percent in off-gas(O2:O2  (%))',
 'Dissolved oxygen concentration(DO2:mg/L)',
 'Vessel Weight(Wt:Kg)',
 'pH(pH:pH)',
 'Temperature(T:K)',
 '2400',
 'Aeration rate(Fg:L/h)',
 'PAA flow(Fpaa:PAA flow (L/h))',
 '2400',
 'Time (h)',
 'Batch reference(Batch_ref:Batch ref)']

In [18]:

df['Aeration rate(Fg:L/h)_moving_avg'] = df['Aeration rate(Fg:L/h)'].rolling(window=100).mean().fillna(method='bfill')
df['pH(pH:pH)_moving_std'] = df['pH(pH:pH)'].rolling(window=100).std().fillna(method='bfill')
df['Temperature(T:K)_moving_std'] = df['pH(pH:pH)'].rolling(window=100).std().fillna(method='bfill')
df['2400_ewm'] = df['2400'].ewm(span=5).mean().fillna(method='bfill') 
df['carbon dioxide percent in off-gas(CO2outgas:%)_moving_avg'] = df['carbon dioxide percent in off-gas(CO2outgas:%)'].rolling(window=60).mean().fillna(method='bfill') 
df['Oxygen in percent in off-gas(O2:O2  (%))_moving_avg'] =  df['Oxygen in percent in off-gas(O2:O2  (%))'].rolling(window=60).mean().fillna(method='bfill')          
df['Dissolved oxygen concentration(DO2:mg/L)_moving_avg'] = df['Dissolved oxygen concentration(DO2:mg/L)'].rolling(window=60).mean().fillna(method='bfill')                   
df['Vessel Weight(Wt:Kg)_moving_avg'] = df['Vessel Weight(Wt:Kg)'].rolling(window=60).mean().fillna(method='bfill')    

input_features_transformed = ['pH(pH:pH)_moving_std','Temperature(T:K)_moving_std','2400_ewm','carbon dioxide percent in off-gas(CO2outgas:%)_moving_avg','Aeration rate(Fg:L/h)_moving_avg',
                             'Oxygen in percent in off-gas(O2:O2  (%))_moving_avg','Dissolved oxygen concentration(DO2:mg/L)_moving_avg',
                             'Vessel Weight(Wt:Kg)_moving_avg']

##### Let us visualize the transformed input features

In [19]:
visualize_features(input_features_transformed,df.groupby('Time (h)').mean())

- ##### We can observe that we have managed to capture the essential information of the pH and temperature, namely the deviations over time.
- ##### For the remaining variables with a less seasonal nature, but with a pronounced trend, we captured the direction of the variable over time with the moving averages.

#### Building target variables

##### We are shifting the target variables from 2 time units in the future to the present, to be able to forecast this value with a regression model. With 2 time units, this gives an operator enough time (about 24 Minutes) to react to the real-time predictions, and to sudden unexpected changes in the direction of the concentration of pennicilin during a batch run, and not 12 hours later

In [20]:
# we are shifting the target measurements 


df['target_pennicilin'] = df['Offline Penicillin concentration(P_offline:P(g L^{-1}))_interpolated'].shift(-2).fillna(method='ffill')

df['target_biomass'] = df['Offline Biomass concentratio(X_offline:X(g L^{-1}))_interpolated'].shift(-2).fillna(method='ffill')

shifted_targets = ['target_pennicilin','target_biomass']

#df.drop(columns=target_features, inplace=True)


In [21]:
input_features_transformed, shifted_targets

(['pH(pH:pH)_moving_std',
  'Temperature(T:K)_moving_std',
  '2400_ewm',
  'carbon dioxide percent in off-gas(CO2outgas:%)_moving_avg',
  'Aeration rate(Fg:L/h)_moving_avg',
  'Oxygen in percent in off-gas(O2:O2  (%))_moving_avg',
  'Dissolved oxygen concentration(DO2:mg/L)_moving_avg',
  'Vessel Weight(Wt:Kg)_moving_avg'],
 ['target_pennicilin', 'target_biomass'])

##### Let us try to use the raw input features first

In [22]:
x_train, x_test, y_train, y_test = prepare_dataset(df, train_fraction=0.76,target_columns=shifted_targets,input_columns=input_features)

In [23]:
x_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 86590 entries, 0 to 86589
Data columns (total 11 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Time (h)                                        86590 non-null  float64
 1   Aeration rate(Fg:L/h)                           86590 non-null  int64  
 2   Dissolved oxygen concentration(DO2:mg/L)        86590 non-null  float64
 3   Vessel Weight(Wt:Kg)                            86590 non-null  float64
 4   pH(pH:pH)                                       86590 non-null  float64
 5   Temperature(T:K)                                86590 non-null  float64
 6   carbon dioxide percent in off-gas(CO2outgas:%)  86590 non-null  float64
 7   PAA flow(Fpaa:PAA flow (L/h))                   86590 non-null  float64
 8   Oxygen in percent in off-gas(O2:O2  (%))        86590 non-null  float64
 9   Batch reference(Batch_ref:Batch ref)   

In [24]:
x_train['Batch reference(Batch_ref:Batch ref)'].min(),x_train['Batch reference(Batch_ref:Batch ref)'].max()

(1, 76)

In [25]:
x_test['Batch reference(Batch_ref:Batch ref)'].min(),x_test['Batch reference(Batch_ref:Batch ref)'].max()

(77, 100)

In [26]:
x_train.tail()

,Time (h),Aeration rate(Fg:L/h),Dissolved oxygen concentration(DO2:mg/L),Vessel Weight(Wt:Kg),pH(pH:pH),Temperature(T:K),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oxygen in percent in off-gas(O2:O2 (%)),Batch reference(Batch_ref:Batch ref),2400
86585,229.2,65,12.113,88499.0,6.4993,297.97,1.6957,6.7193,0.19004,76,167010.0
86586,229.4,65,12.146,88577.0,6.4974,297.90,1.6961,6.7198,0.19005,76,167660.0
86587,229.6,65,12.157,88656.0,6.4974,298.03,1.6966,6.7124,0.19000,76,167500.0
86588,229.8,65,12.152,88735.0,6.5003,297.96,1.6971,6.6917,0.18992,76,167940.0
86589,230.0,65,12.174,88814.0,6.5040,297.89,1.6975,6.6609,0.18995,76,168680.0


In [27]:
x_test.head()

,Time (h),Aeration rate(Fg:L/h),Dissolved oxygen concentration(DO2:mg/L),Vessel Weight(Wt:Kg),pH(pH:pH),Temperature(T:K),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oxygen in percent in off-gas(O2:O2 (%)),Batch reference(Batch_ref:Batch ref),2400
86590,0.2,30,14.713,62364.0,6.5586,297.66,0.083954,5.0,0.20798,77,0.0
86591,0.4,30,14.685,62366.0,6.5375,297.88,0.095033,5.0,0.20679,77,0.0
86592,0.6,30,14.678,62368.0,6.5173,297.86,0.098748,5.0,0.20647,77,0.0
86593,0.8,30,14.653,62369.0,6.4978,298.15,0.101010,5.0,0.20622,77,0.0
86594,1.0,30,14.640,62371.0,6.4813,298.04,0.103040,5.0,0.20609,77,0.0


In [50]:
#y_test = pd.DataFrame(y_test)
y_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27345 entries, 86590 to 113934
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   target_pennicilin  27345 non-null  float64
 1   target_biomass     27345 non-null  float64
dtypes: float64(2)
memory usage: 427.4 KB


In [51]:
input_features

['carbon dioxide percent in off-gas(CO2outgas:%)',
 'Oxygen in percent in off-gas(O2:O2  (%))',
 'Dissolved oxygen concentration(DO2:mg/L)',
 'Vessel Weight(Wt:Kg)',
 'pH(pH:pH)',
 'Temperature(T:K)',
 '2400',
 'Aeration rate(Fg:L/h)',
 'PAA flow(Fpaa:PAA flow (L/h))',
 '2400',
 'Time (h)',
 'Batch reference(Batch_ref:Batch ref)']

##### We are taking batch runs 1-76 as the training dataset, and batch runs 77-100 as the test dataset.
##### We also checked if the datasets were starting not at the beginning of a batch run, or not ending at the end of a batch run. that is also not the case

In [29]:
%%time
model = RF(n_estimators=100,n_jobs=-1)
model.fit(x_train.drop(['Time (h)','Batch reference(Batch_ref:Batch ref)'],axis=1), y_train)
y_pred_original = model.predict(x_test.drop(['Time (h)','Batch reference(Batch_ref:Batch ref)'],axis=1))

CPU times: total: 2min 24s
Wall time: 14.2 s


##### We are comparing the metrics for the different targets

In [30]:
for n,i in enumerate(shifted_targets):
    metrics_base = compute_metrics(y_test, y_pred_original[:,n],i)
    print(metrics_base)

{'target': 'target_pennicilin', 'MAE': 0.23, 'RMSE': 0.77, 'MAPE': 2.9521948940419258e+22}
{'target': 'target_biomass', 'MAE': 0.48, 'RMSE': 0.93, 'MAPE': 3.06}


#### now let us try the transformed features instead and compare both performances

In [31]:
x_train, x_test, y_train, y_test = prepare_dataset(df, train_fraction=0.76,target_columns=shifted_targets,input_columns= input_features +  input_features_transformed)



In [32]:
%%time
model = RF(n_estimators=100,n_jobs=-1)
model.fit(x_train.drop(['Time (h)','Batch reference(Batch_ref:Batch ref)'],axis=1), y_train)
y_pred_transformed = model.predict(x_test.drop(['Time (h)','Batch reference(Batch_ref:Batch ref)'],axis=1))

for n,i in enumerate(shifted_targets):
    print("transformed feature metrics")
    metrics_base = compute_metrics(y_test, y_pred_transformed[:,n],i)
    print(metrics_base)


transformed feature metrics
{'target': 'target_pennicilin', 'MAE': 0.18, 'RMSE': 0.7, 'MAPE': 5.777235879084813e+21}
transformed feature metrics
{'target': 'target_biomass', 'MAE': 0.51, 'RMSE': 0.94, 'MAPE': 3.0}
CPU times: total: 5min 32s
Wall time: 31.7 s


##### there seems to be an improvement for the target pennicillin, but not for the target biomass

##### let us compare both metrics to the standard deviation present in the test dataset

In [33]:
y_statistics = y_test[shifted_targets].describe().transpose()

y_statistics


,count,mean,std,min,25%,50%,75%,max
target_pennicilin,27345.0,13.361292,9.848887,3.694700e-26,4.960912,11.39735,22.364050,33.167
target_biomass,27345.0,18.649244,6.423652,3.938100e-01,15.897800,20.97260,23.423083,26.732


##### the standard deviation in the test dataset lies at 9.84 for the pennicilin target, and 6.42 for the biomass target. Both of our metrics fall well below this value, accounting for about 10% of the existing standard deviation present in the test dataset

#### Visualizing the two predictions

In [34]:
for n,i in enumerate(shifted_targets):

    y_test[f'Predictions_{i}_original'] = y_pred_original[:,n]
    y_test[f'Predictions_{i}_transformed'] = y_pred_transformed[:,n]


y_test['Time (h)'] = x_test['Time (h)']

y_test_grouped = y_test.groupby('Time (h)').mean()

fig = go.Figure()


# Add first time series
fig.add_trace(go.Scatter(
    x=y_test_grouped.index, y=y_test_grouped["Predictions_target_pennicilin_original"],
    mode="lines",
    name="Pennicilin predicted original features",
    line=dict(color="blue")
))

fig.add_trace(go.Scatter(
    x=y_test_grouped.index, y=y_test_grouped["Predictions_target_pennicilin_transformed"],
    mode="lines",
    name="Pennicilin predicted transformed features",
    line=dict(color="purple")
))

# Add second time series
fig.add_trace(go.Scatter(
    x=y_test_grouped.index, y=y_test_grouped["target_pennicilin"],
    mode="lines",
    name="Pennicilin true",
    line=dict(color="red")
))

# Customize layout
fig.update_layout(
    title="Predicted and target values over time",
    xaxis_title="Time in 12 Minutes Intervals",
    yaxis_title="Feature value",
    legend=dict(x=0, y=1),
    template="plotly_white"
)

fig.show()

**Observations**

- We can observe that the predictions are able to replicate the steady peak towards a maximum pennicilin concentration, as in the original interpolated measurements.

- We see that beyond this peak, the original measurements become more erratic, and the predictions are not able to capture this quite as well. However, we are more interested in capturing the behaviour leading upto and around the peak

**Simple model**

We are going to compare our model with a simple baseline model, which predicts the next value as a simple moving average of the prior target.

In [35]:
y_dummy = y_test[shifted_targets].rolling(window=6).mean().fillna(method='bfill')


In [36]:
# we are comparing the dummy model for the colum target_pennicilin (offline pennicilin concentration measurements) as an example

for n,i in enumerate(shifted_targets):
    metrics_base = compute_metrics(y_test, y_pred_transformed[:,n],i)
    metrics_dummy = compute_metrics(y_test, y_dummy[i],i)
    print('Dummy model scores:', metrics_dummy)
    print('Baseline model scores:', metrics_base)

Dummy model scores: {'target': 'target_pennicilin', 'MAE': 0.11, 'RMSE': 0.85, 'MAPE': 1.5165054263388383e+24}
Baseline model scores: {'target': 'target_pennicilin', 'MAE': 0.18, 'RMSE': 0.7, 'MAPE': 5.777235879084813e+21}
Dummy model scores: {'target': 'target_biomass', 'MAE': 0.1, 'RMSE': 0.75, 'MAPE': 4.83}
Baseline model scores: {'target': 'target_biomass', 'MAE': 0.51, 'RMSE': 0.94, 'MAPE': 3.0}


***We see that our Baseline Model still marginally outperforms our model on the target biomass, but not on the target pennicilin. Here we managed to achieve a better RMSE score compared to the dummy model, and comparable MAE scores as well.***



# Baseline Model with Target Lag Features

A good idea is to add the lagged values of the target because we assume it's going to be available by the time we make the predictions for the next time interval. From the previous plots, we could see that the measurements peak between hours 205 and 210 of the aggregated batch runs. Let's try to capture the meaningful lagged features from this time interval.



In [37]:
df['index'] = df.index
df['batch_ref'] = batch_references # we are using the batch reference we saved early in the script

# Baseline Model with Input Lag & Delta Features

Let's add the lag features for other features, not only for the target.

We will add only the first 5 lags, having an autocorrelation between 0.7 and 0.9

In [38]:
# we are adding the difference of the feature with its lagged self here
delta_feats = input_features[:-2]

delta_cols = []
n = 5
for feat in delta_feats:
    for i in range(1, n+1):
        df[f'{feat}_delta'] = df[feat].shift(i).fillna(method='bfill')

        delta_cols.append(f'{feat}_delta')


In [39]:
# with this code, we are aiming to get the lags with autocorrelated scores of at least 0.7 for each batch using the dictionary created in the previous section.
# the input features having varying degrees of 
best_lags = {}
lags = []
counter = 0

for feat in input_features[:-2]:
    for lag in range(1,100,1):
        score = df[feat].autocorr(lag=lag)
        if score > 0.7 and score < 0.9:
            lags.append(score)
        best_lags[feat] = lags[:5]



In [40]:
best_lags

{'carbon dioxide percent in off-gas(CO2outgas:%)': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'Oxygen in percent in off-gas(O2:O2  (%))': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'Dissolved oxygen concentration(DO2:mg/L)': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'Vessel Weight(Wt:Kg)': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'pH(pH:pH)': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'Temperature(T:K)': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 '2400': [0.8944957607396802,
  0.8881481141238777,
  0.8817960800680872,
  0.8754454980821127,
  0.8691150519815765],
 'Aeration rate(Fg:L

In [41]:
lag_feats = input_features[:-2]
lag_cols_feature = []
n = 5
for feat in lag_feats:
    for i in range(1, n+1):
        df[f'{feat}_lag_{i}'] = df[feat].shift(i).fillna(method='bfill')

        lag_cols_feature.append(f'{feat}_lag_{i}')

##### we unfortunately cannot use the lag values of the target, as we only obtain this in 12 hour intervals.
##### using interpolated target values as a lag source would not be very accurate, nor realisitic in a production setting

#### Dataframe with lagged and delta features, and lagged targets

In [42]:
df[input_features + input_features_transformed +lag_cols_feature+delta_cols+shifted_targets].columns

Index(['carbon dioxide percent in off-gas(CO2outgas:%)',
       'Oxygen in percent in off-gas(O2:O2  (%))',
       'Dissolved oxygen concentration(DO2:mg/L)', 'Vessel Weight(Wt:Kg)',
       'pH(pH:pH)', 'Temperature(T:K)', '2400', 'Aeration rate(Fg:L/h)',
       'PAA flow(Fpaa:PAA flow (L/h))', '2400',
       ...
       'PAA flow(Fpaa:PAA flow (L/h))_delta',
       'PAA flow(Fpaa:PAA flow (L/h))_delta',
       'PAA flow(Fpaa:PAA flow (L/h))_delta', '2400_delta', '2400_delta',
       '2400_delta', '2400_delta', '2400_delta', 'target_pennicilin',
       'target_biomass'],
      dtype='object', length=122)

In [43]:
(x_train, x_test, y_train,y_test) = prepare_dataset(df,target_columns=shifted_targets,
                                                   input_columns=input_features[:-2] + lag_cols_feature + delta_cols + input_features_transformed,
                                                   train_fraction=0.76)

In [44]:
x_train.head(10)

,Aeration rate(Fg:L/h),Dissolved oxygen concentration(DO2:mg/L),Vessel Weight(Wt:Kg),pH(pH:pH),Temperature(T:K),carbon dioxide percent in off-gas(CO2outgas:%),PAA flow(Fpaa:PAA flow (L/h)),Oxygen in percent in off-gas(O2:O2 (%)),2400,Aeration rate(Fg:L/h)_moving_avg,...,Aeration rate(Fg:L/h)_lag_1,Aeration rate(Fg:L/h)_lag_2,Aeration rate(Fg:L/h)_lag_3,Aeration rate(Fg:L/h)_lag_4,Aeration rate(Fg:L/h)_lag_5,PAA flow(Fpaa:PAA flow (L/h))_lag_1,PAA flow(Fpaa:PAA flow (L/h))_lag_2,PAA flow(Fpaa:PAA flow (L/h))_lag_3,PAA flow(Fpaa:PAA flow (L/h))_lag_4,PAA flow(Fpaa:PAA flow (L/h))_lag_5
0,30,14.711,62574.0,6.4472,298.22,0.089514,5.0,0.19595,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
1,30,14.699,62585.0,6.4932,298.17,0.101760,5.0,0.20390,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
2,30,14.686,62598.0,6.5425,298.14,0.105800,5.0,0.20575,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
3,30,14.661,62607.0,6.5753,298.11,0.108190,5.0,0.20602,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
4,30,14.633,62613.0,6.5825,298.09,0.110300,5.0,0.20589,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
5,30,14.621,62617.0,6.5717,298.08,0.112420,5.0,0.20580,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
6,30,14.604,62620.0,6.5521,298.07,0.114630,5.0,0.20569,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
7,30,14.583,62622.0,6.5287,298.06,0.116940,5.0,0.20553,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
8,30,14.547,62624.0,6.5061,298.06,0.119360,5.0,0.20524,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0
9,30,14.529,62626.0,6.4844,298.05,0.121890,5.0,0.20507,0.0,37.2,...,30.0,30.0,30.0,30.0,30.0,5.0,5.0,5.0,5.0,5.0


In [45]:
x_train.columns

Index(['Aeration rate(Fg:L/h)', 'Dissolved oxygen concentration(DO2:mg/L)',
       'Vessel Weight(Wt:Kg)', 'pH(pH:pH)', 'Temperature(T:K)',
       'carbon dioxide percent in off-gas(CO2outgas:%)',
       'PAA flow(Fpaa:PAA flow (L/h))',
       'Oxygen in percent in off-gas(O2:O2  (%))', '2400',
       'Aeration rate(Fg:L/h)_moving_avg', 'pH(pH:pH)_moving_std',
       'Temperature(T:K)_moving_std', '2400_ewm',
       'carbon dioxide percent in off-gas(CO2outgas:%)_moving_avg',
       'Oxygen in percent in off-gas(O2:O2  (%))_moving_avg',
       'Dissolved oxygen concentration(DO2:mg/L)_moving_avg',
       'Vessel Weight(Wt:Kg)_moving_avg',
       'carbon dioxide percent in off-gas(CO2outgas:%)_delta',
       'Oxygen in percent in off-gas(O2:O2  (%))_delta',
       'Dissolved oxygen concentration(DO2:mg/L)_delta',
       'Vessel Weight(Wt:Kg)_delta', 'pH(pH:pH)_delta',
       'Temperature(T:K)_delta', '2400_delta', 'Aeration rate(Fg:L/h)_delta',
       'PAA flow(Fpaa:PAA flow (L/h))_delt

##### now, let us retrain the model with the engineered lag and delta features, to see if we could improve it's performance

In [46]:
%%time
np.random.seed(23)
model = RF(n_estimators=100, verbose=1,n_jobs=-1)
model.fit(x_train, y_train)
y_pred_lag_delta = model.predict(x_test)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:   33.6s


CPU times: total: 19min 7s
Wall time: 1min 50s


[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  1.8min finished
[Parallel(n_jobs=12)]: Using backend ThreadingBackend with 12 concurrent workers.
[Parallel(n_jobs=12)]: Done  26 tasks      | elapsed:    0.0s
[Parallel(n_jobs=12)]: Done 100 out of 100 | elapsed:    0.0s finished


In [47]:
category = ['original','transformed','lag&delta']
for n,i in enumerate(shifted_targets):
    for c,y in zip(category,[y_pred_original,y_pred_transformed,y_pred_lag_delta]):
        print(f"{c} feature metrics")
        metrics_base = compute_metrics(y_test, y[:,n],i)
        print(metrics_base)








original feature metrics
{'target': 'target_pennicilin', 'MAE': 0.23, 'RMSE': 0.77, 'MAPE': 2.9521948940419258e+22}
transformed feature metrics
{'target': 'target_pennicilin', 'MAE': 0.18, 'RMSE': 0.7, 'MAPE': 5.777235879084813e+21}
lag&delta feature metrics
{'target': 'target_pennicilin', 'MAE': 0.18, 'RMSE': 0.69, 'MAPE': 1.0575453271662458e+20}
original feature metrics
{'target': 'target_biomass', 'MAE': 0.48, 'RMSE': 0.93, 'MAPE': 3.06}
transformed feature metrics
{'target': 'target_biomass', 'MAE': 0.51, 'RMSE': 0.94, 'MAPE': 3.0}
lag&delta feature metrics
{'target': 'target_biomass', 'MAE': 0.5, 'RMSE': 0.93, 'MAPE': 2.82}


We see that adding the feature & target lags and their deltas further improved the performance with the target pennicilin, as compared to using the input features alone.

For the target biomass, this did not lead to a significant improvement, nor to a significant decrease.

We will keep the additional features we generated


# Gradient Boosting with Hyperparameters Tuning

**we are going to offload the parameter tuning workload to Vertex AI by Google Cloud Platform, as for dataset of this size, it is not feasible to run this locally**

**The model training/ retraining workload will be offloaded as well, as this can be very time consuming as the dataset grows, and model retraining will need to be conducted on the go, as soon as a certain treshold is reached, before the next batches can be run and its predictions can be relied upon.**

**we will end this notebook by saving the model.**

In [49]:
# Save the model to experiments
import joblib
joblib.dump(model,'./models/experiments/trained_model.joblib')

['./models/experiments/trained_model.joblib']

# Conclusions

- The Baseline Random Forest model with no feature engineering gave almost the same result as the dummy model (last value shifted forward)

- Adding the target lags and feature lags improved the model performance slightly.

- For the production pipeline, we will run the model training and hyperparameter tuning workloads in the cloud, to make use of the increased compute resources. Already with about 100k rows, the training and tuning times are not trivial.

- with a model predicting pennicilin concentrations two timesteps ahead, this gives operators enough time to adjust their PAA and Substrate flow rates, to achieve the desired end pennicilin concentration levels.

- this model also eliminates the need to rely on the very delayed pennicilin measurements being available every 12 hours. Operators are able to check in real time if the batch run is proceeding according to plan/schedule, and intervene if irregular pennicilin concentrations growth directions are predicted.

- with data from field trials of the model, we will be able to determine when the pennicilin concentration reaches a peak as well. At the moment, only with interpolated target values, this is not possible to predict accurately.